In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

In [2]:
input_str = 'apple'
label_str = 'pple!'

char_vocab = sorted(list(set(input_str+label_str)))
vocab_size = len(char_vocab)
print('문자 집합의 크기: {}'.format(vocab_size))

문자 집합의 크기: 5


In [3]:
input_size = vocab_size
hidden_size = 5
output_size = 5
learning_rate = 0.1

In [4]:
char2index = dict((c, i) for i, c in enumerate(char_vocab))
print(char2index)

{'!': 0, 'a': 1, 'e': 2, 'l': 3, 'p': 4}


In [5]:
index2char = {}
for key, value in char2index.items():
  index2char[value] = key
print(index2char)

{0: '!', 1: 'a', 2: 'e', 3: 'l', 4: 'p'}


In [6]:
x_data = [char2index[c] for c in input_str]
y_data = [char2index[c] for c in label_str]
print(x_data)
print(y_data)

[1, 4, 4, 3, 2]
[4, 4, 3, 2, 0]


In [7]:
# 배치 차원 추가
# 텐서 연산인 unsqueeze(0)를 통해 해결할 수도 있었음.
x_data = [x_data]
y_data = [y_data]
print(x_data)
print(y_data)

[[1, 4, 4, 3, 2]]
[[4, 4, 3, 2, 0]]


In [8]:
x_one_hot = [np.eye(vocab_size)[x] for x in x_data]
print(x_one_hot)

[array([[0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.]])]


In [9]:
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

<ipython-input-9-c1bfbd518a63>:1: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:275.)
  X = torch.FloatTensor(x_one_hot)


In [10]:
print('훈련 데이터의 크기 : {}'.format(X.shape))
print('레이블의 크기 : {}'.format(Y.shape))

훈련 데이터의 크기 : torch.Size([1, 5, 5])
레이블의 크기 : torch.Size([1, 5])


In [11]:
class Net(torch.nn.Module):
  def __init__(self, input_size, hidden_size, output_size):
    super(Net, self).__init__()
    self.rnn = torch.nn.RNN(input_size, hidden_size, batch_first=True)
    self.fc = torch.nn.Linear(hidden_size, output_size)

  def forward(self, x):
    x, _status = self.rnn(x)
    x = self.fc(x)
    return x

In [12]:
net = Net(input_size, hidden_size, output_size)

In [13]:
outputs = net(X)
print(outputs.shape)

torch.Size([1, 5, 5])


In [14]:
print(outputs.view(-1,input_size).shape)

torch.Size([5, 5])


In [15]:
print(Y.shape)
print(Y.view(-1).shape)

torch.Size([1, 5])
torch.Size([5])


In [16]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), learning_rate)

In [17]:
for i in range(100):
    optimizer.zero_grad()
    outputs = net(X)
    loss = criterion(outputs.view(-1, input_size), Y.view(-1)) # view를 하는 이유는 Batch 차원 제거를 위해
    loss.backward() # 기울기 계산
    optimizer.step() # 아까 optimizer 선언 시 넣어둔 파라미터 업데이트

    # 아래 세 줄은 모델이 실제 어떻게 예측했는지를 확인하기 위한 코드.
    result = outputs.data.numpy().argmax(axis=2) # 최종 예측값인 각 time-step 별 5차원 벡터에 대해서 가장 높은 값의 인덱스를 선택
    result_str = ''.join([index2char[c] for c in np.squeeze(result)])
    print(i, "loss: ", loss.item(), "prediction: ", result, "true Y: ", y_data, "prediction str: ", result_str)

0 loss:  1.8321056365966797 prediction:  [[3 1 3 3 3]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  lalll
1 loss:  1.5650060176849365 prediction:  [[3 3 3 3 3]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  lllll
2 loss:  1.3789294958114624 prediction:  [[4 4 0 4 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pp!p!
3 loss:  1.2396681308746338 prediction:  [[4 4 0 4 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pp!p!
4 loss:  1.120923638343811 prediction:  [[4 4 0 4 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pp!p!
5 loss:  0.9983231425285339 prediction:  [[4 4 0 4 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pp!p!
6 loss:  0.8612934350967407 prediction:  [[4 4 0 4 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pp!p!
7 loss:  0.696083128452301 prediction:  [[4 4 0 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pp!e!
8 loss:  0.5429848432540894 prediction:  [[4 4 3 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pple!
9 loss:  0.45836544036865234 prediction:  [[4 4 3 2 3]] t